In [2]:
import numpy as np
import visual_bow as bow
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.externals import joblib
import glob
import random
import warnings

SCORING = 'f1_micro'
print 'Scoring grid search with metric: %s' % SCORING

OpenCV VERSION (should be 3.1.0 or later, with nonfree modules installed!): 3.1.0-dev
Scoring grid search with metric: f1_micro


C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# Get all possible negative images and label them False
positive_folder='panda'
all_negs = [(path, False) for path in bow.neg_img_cal101(positive_folder)]
print '%i total negative imgs to choose from' % len(all_negs)
print all_negs[:5]

8639 total negative imgs to choose from
[('101_ObjectCategories\\accordion\\image_0001.jpg', False), ('101_ObjectCategories\\accordion\\image_0002.jpg', False), ('101_ObjectCategories\\accordion\\image_0003.jpg', False), ('101_ObjectCategories\\accordion\\image_0004.jpg', False), ('101_ObjectCategories\\accordion\\image_0005.jpg', False)]


In [4]:
# Get all the positive images you have (in the panda_rip folder) and label them True
positive_imgs = [(path, True) for path in glob.glob('panda_rip/*')]
print '%i positive images' % len(positive_imgs)
print positive_imgs[:5]

38 positive images
[('panda_rip\\image_0001.jpg', True), ('panda_rip\\image_0002.jpg', True), ('panda_rip\\image_0003.jpg', True), ('panda_rip\\image_0004.jpg', True), ('panda_rip\\image_0005.jpg', True)]


In [5]:
# take N random negative images, where N is no of positive images
# then concatenate N pos + N neg and shuffle.
chosen_negs = random.sample(all_negs, len(positive_imgs))
imgs = chosen_negs + positive_imgs

np.random.shuffle(imgs)

print '%i total images (1:1 positive:negative)' % len(imgs)
print imgs[:5]

76 total images (1:1 positive:negative)
[('101_ObjectCategories\\revolver\\image_0028.jpg', False), ('panda_rip\\image_0022.jpg', True), ('panda_rip\\image_0009.jpg', True), ('panda_rip\\image_0004.jpg', True), ('101_ObjectCategories\\airplanes\\image_0235.jpg', False)]


In [6]:
%%time

img_descs, y = bow.gen_sift_features(imgs)

generating SIFT descriptors for 76 images
SIFT descriptors generated.
Wall time: 4.64 s


In [36]:
# joblib.dump(img_descs, 'pickles/img_descs/img_descs.pickle')
# joblib.dump(y, 'pickles/img_descs/y.pickle')

['pickles/img_descs/y.pickle', 'pickles/img_descs/y.pickle_01.npy']

In [7]:
# generate indexes for train/test/val split
training_idxs, test_idxs, val_idxs = bow.train_test_val_split_idxs(
    total_rows=len(imgs), 
    percent_test=0.15, 
    percent_val=0.15
)

Train-test-val split: 54 training rows, 11 test rows, 11 validation rows


# Cluster the SIFT descriptors

In [8]:
%%time

K_CLUSTERS = 250

# MiniBatchKMeans annoyingly throws tons of deprecation warnings that fill up the notebook. Ignore them.
warnings.filterwarnings('ignore')

X, cluster_model = bow.cluster_features(
    img_descs, 
    training_idxs=training_idxs, 
    cluster_model=MiniBatchKMeans(n_clusters=K_CLUSTERS)
)

warnings.filterwarnings('default')

X_train, X_test, X_val, y_train, y_test, y_val = bow.perform_data_split(X, y, training_idxs, test_idxs, val_idxs)

23027 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=250, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 250 words
done clustering. Using clustering model to generate BoW histograms for each image.
done generating BoW histograms.
Wall time: 2.39 s


## Uncomment to pickle the clustered Visual BoW features

In [32]:
# for obj, obj_name in zip( [X_train, X_test, X_val, y_train, y_test, y_val], 
#                          ['X_train', 'X_test', 'X_val', 'y_train', 'y_test', 'y_val'] ):
#     joblib.dump(obj, 'pickles/feature_data/%s.pickle' % obj_name)

## Uncomment to LOAD pickle of clustered Visual BoW features

In [ ]:
# for obj_name in ['X_train', 'X_test', 'X_val', 'y_train', 'y_test', 'y_val']:
#     exec("{obj_name} = joblib.load('pickles/feature_data/{obj_name}.pickle')".format(obj_name=obj_name))
#     exec("print obj_name, len({0})".format(obj_name))

# Classify with SVM

In [9]:
%%time

# c_vals = [0.0001, 0.01, 0.1, 1, 10, 100, 1000]
c_vals = [0.1, 1, 5, 10]
# c_vals = [1]

gamma_vals = [0.5, 0.1, 0.01, 0.0001, 0.00001]
# gamma_vals = [0.5, 0.1]
# gamma_vals = [0.1]

param_grid = [
  {'C': c_vals, 'kernel': ['linear']},
  {'C': c_vals, 'gamma': gamma_vals, 'kernel': ['rbf']},
 ]

svc = GridSearchCV(SVC(), param_grid, n_jobs=-1, scoring=SCORING)
svc.fit(X_train, y_train)
print 'train score (%s):'%SCORING, svc.score(X_train, y_train)
print 'test score (%s):'%SCORING, svc.score(X_test, y_test)

print svc.best_estimator_

train score (f1_micro): 1.0
test score (f1_micro): 0.727272727273
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Wall time: 3.43 s


C:\Anaconda2\lib\site-packages\sklearn\grid_search.py:426: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)


# We have our estimator, this is how it could classify random pictures

In [10]:
for img_path, label in random.sample(all_negs, 10):
    print img_path, svc.predict(bow.img_to_vect(img_path, cluster_model))

101_ObjectCategories\Faces_easy\image_0316.jpg ['False']
101_ObjectCategories\car_side\image_0093.jpg ['False']
101_ObjectCategories\garfield\image_0002.jpg ['False']
101_ObjectCategories\Motorbikes\image_0280.jpg ['False']
101_ObjectCategories\hawksbill\image_0012.jpg ['True']
101_ObjectCategories\revolver\image_0018.jpg ['False']
101_ObjectCategories\airplanes\image_0139.jpg ['False']
101_ObjectCategories\Motorbikes\image_0117.jpg ['False']
101_ObjectCategories\airplanes\image_0215.jpg ['False']
101_ObjectCategories\brain\image_0027.jpg ['False']


## Uncomment to pickle the best SVC classifier & kmeans

In [11]:
# joblib.dump(svc.best_estimator_, 'pickles/svc/svc.pickle')
# joblib.dump(cluster_model, 'pickles/cluster_model/cluster_model.pickle')

# Try AdaBoost, it's a common choice for SIFT features

In [12]:
%%time

ada_params = {
    'n_estimators':[100, 250, 500, 750],
    'learning_rate':[0.8, 0.9, 1.0, 1.1, 1.2]
}

# ada = AdaBoostClassifier(n_estimators=MAX_ESTIMATORS, learning_rate=0.8)
ada = GridSearchCV(AdaBoostClassifier(), ada_params, n_jobs=-1, scoring=SCORING)
ada.fit(X_train, y_train)
print 'train score (%s):'%SCORING, ada.score(X_train, y_train)
print 'test score (%s):'%SCORING, ada.score(X_test, y_test)
print ada.best_estimator_

train score (f1_micro): 1.0
test score (f1_micro): 0.727272727273
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.9, n_estimators=250, random_state=None)
Wall time: 13.6 s


## Uncomment to pickle the AdaBoostClassifier

In [13]:
# joblib.dump(ada.best_estimator_, 'pickles/ada/ada.pickle');
# print 'picked adaboost'

# TODO

* Separate out the clustering from the feature generation. They should be 2 different functions, the clustering should take the SIFT **training** data as an argument. It has labels already, right? Then you can save the SIFT data before clustering. Finally, you can do a grid search across K_CLUSTERS.

* Also it would be cool to graph the above.